In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, IntegerType
from pyspark.sql.functions import from_json, to_json, col, struct
import time
import signal
from typing import Union

from tools_kafka import Kafka
from tools_pyspark_hdfs import Spark_HDFS as HDFS
from tools_pyspark import stop_all_streams, sink, console_output

def read_stream(spark, topic_name, schema, kf):
    """Возвращает стрим топика кафки"""
    return spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kf.SERVERS) \
        .option("subscribe", topic_name) \
        .option("startingOffsets", "earliest") \
        .option("maxOffsetsPerTrigger", "5") \
        .load() \
        .select(F.from_json(F.col("value").cast("String"), schema).alias("value"), "offset") \
        .select("value.*", "offset")

spark = SparkSession.builder.appName("my_spark").getOrCreate()

kf = Kafka()
hdfs = HDFS(spark)

topic_name = 'lesson3'
sink_path = f'tmp_{topic_name}'  # в случае с kafka это название топика, где хранится информация
checkpoint = f'sink_{sink_path}'

schema = StructType() \
    .add("column_1", StringType()) \
    .add("column_2", IntegerType())

kf.add('lesson3') # создаём топик кафки

Топик успешно создан: lesson3


True

In [4]:
# Читаем файл и переносим его в топик кафки
stream = spark \
    .readStream \
    .format("csv") \
    .option("header", True) \
    .option("maxFilesPerTrigger", 1) \
    .schema(schema) \
    .csv("/data/") \
    .selectExpr("CAST(null AS STRING) as key", 
                "CAST(to_json(struct(*)) AS STRING) as value") \
    .writeStream \
    .format("kafka") \
    .outputMode("append") \
    .option("kafka.bootstrap.servers", kf.SERVERS) \
    .option("topic", topic_name) \
    .option("checkpointLocation", "checkpoints/stream_read_write") \
    .start()

# выводим содержимое топика
kf.get(topic_name) 
# остановим потоки
stop_all_streams(spark) 

{"column_1":"a","column_2":2}
{"column_1":"b","column_2":4}
{"column_1":"c","column_2":8}
Stopping stream: <pyspark.sql.streaming.StreamingQuery object at 0x7d604ef353c0>


### Попробуем сохранить данные в sink тремя разными способами: memory, kafka (+json), parquet

In [5]:
# читаем топик кафки
df = read_stream(spark, topic_name, schema, kf)
# записываем в sink - parquet
df = sink(df, path=sink_path, form='parquet', checkpoint=checkpoint)
# немного подождём - процесс записи в hdfs небыстрый
time.sleep(5)
# проверяем sink в parquet:
files = hdfs.ls(sink_path, return_paths=True, recursive=True)
# проверяем sink в parquet:
for file in files:
    if file.endswith('.parquet'):
        hdfs.cat(file)
# останавливаем потоки
stop_all_streams(spark)

Список файлов в директории tmp_lesson3:
hdfs://localhost/user/root/tmp_lesson3/_spark_metadata/0
hdfs://localhost/user/root/tmp_lesson3/part-00000-1a050473-7ac2-4ed3-b083-f8c1e8aa65b0-c000.snappy.parquet
Содержимое файла hdfs://localhost/user/root/tmp_lesson3/part-00000-1a050473-7ac2-4ed3-b083-f8c1e8aa65b0-c000.snappy.parquet:
PAR1 *.¯ÎÄÙ
   P      a   b   c $(ÈÝò   D            <8ÙÄöÓ       
<              ac                           \   dV   ºf   LHspark_schema %column_1% L   %column_2 %offset <5 column_1X\<6 (ca     ® " &d5 column_2RV&d<       (           ÂÂ. &º5 offsetjf&º<                                    Öð>   ,org.apache.spark.version3.3.1 )org.apache.spark.sql.parquet.row.metadataß{"type":"struct","fields":[{"name":"column_1","type":"string","n

In [6]:
# читаем топик кафки
df = read_stream(spark, topic_name, schema, kf)
# записываем в sink - kafka
df = sink(df, path=sink_path, form='kafka', checkpoint=sink_path)
# проверяем sink в kafka:
kf.get(name=sink_path, from_beginning=True)
# останавливаем потоки
stop_all_streams(spark)

Такого топика не существует: tmp_lesson3
Создаём топик: tmp_lesson3
Топик успешно создан: tmp_lesson3
{a, 2, 0}
{b, 4, 1}
{c, 8, 2}
Stopping stream: <pyspark.sql.streaming.StreamingQuery object at 0x7d604e612320>


In [7]:
# читаем топик кафки
df = read_stream(spark, topic_name, schema, kf)
# записываем в sink - kafka
df = sink(df, path=sink_path+'_json', form='kafka', checkpoint=sink_path+'_json', json=True)
# проверяем sink в kafka:
kf.get(name=sink_path+'_json', from_beginning=True)
# останавливаем потоки
stop_all_streams(spark)

Такого топика не существует: tmp_lesson3_json
Создаём топик: tmp_lesson3_json
Топик успешно создан: tmp_lesson3_json
{"column_1":"a","column_2":2,"offset":0}
{"column_1":"b","column_2":4,"offset":1}
{"column_1":"c","column_2":8,"offset":2}
Stopping stream: <pyspark.sql.streaming.StreamingQuery object at 0x7d604e612a10>


In [8]:
# читаем топик кафки
df = read_stream(spark, topic_name, schema, kf)
# записываем в sink - memory
df = sink(df, path=sink_path, form='memory')
# проверяем sink в memory:
time.sleep(5)
spark.sql(f"select * from {sink_path}").show()
# останавливаем потоки
stop_all_streams(spark)

+--------+--------+------+
|column_1|column_2|offset|
+--------+--------+------+
|       a|       2|     0|
|       b|       4|     1|
|       c|       8|     2|
+--------+--------+------+

Stopping stream: <pyspark.sql.streaming.StreamingQuery object at 0x7d604ef37760>


In [9]:
# убираем за собой
stop_all_streams(spark)
kf.rm(topic_name) # удаляем топики кафки
kf.rm(sink_path)
kf.rm(sink_path+'_json')
hdfs.rm('/tmp') # чистим временные данные
hdfs.rm('/user/root') # чистим чекпойнты

Топик успешно удален: lesson3
Топик успешно удален: tmp_lesson3
Топик успешно удален: tmp_lesson3_json
Файл успешно удалён: /tmp
Файл успешно удалён: /user/root


True